# Working with existing deployments

In [1]:
from skil import Skil, Service
from skil import get_workspace_by_id
from skil import get_experiment_by_id
from skil import get_model_by_id
from skil import get_deployment_by_id

skil_server = Skil(
    host      = 'localhost',
    port      = 9008,
    user_id   = 'admin',
    password  = 'Skymind'
)

Using TensorFlow backend.


'>>> Authenticating SKIL...'
'>>> Done!'


## Get deployment

In [4]:
workspace_id  = 'd2a9f40d-69b0-4497-8949-f0d295954a16'
experiment_id = 'yolo-experiment-01'
model_id      = 'yolo-model-01'
deployment_id = '0'

work_space = get_workspace_by_id(skil_server, workspace_id)

experiment = get_experiment_by_id(work_space, experiment_id)

deployment = get_deployment_by_id(skil_server, deployment_id)

model = get_model_by_id(experiment, model_id)

## Start service

In [5]:
service = Service(
    skil=skil_server,
    model=model,
    deployment=deployment,
    model_deployment=None
)

## Run YOLO-v2 Detection Inference

In [10]:
import skil, requests, cv2, json
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline

# Set configurations & temp image
threshold=0.5
needs_preprocessing=False
image_path='images/img-4.jpg'


url = 'http://{}/endpoints/{}/model/{}/v{}/detectobjects'.format(
    skil_server.config.host,
    deployment.name,
    model.name,
    model.version
)

response   = requests.post(
    url    = url,
    headers= skil_server.auth_headers,
    files  = {'file': open(image_path,'rb').read()},
    data   = {
        'id': model.id,
        'needs_preprocessing':needs_preprocessing,
        'threshold': threshold
    }
)
detections = response.json()
print(json.dumps(detections['objects'][0], indent=4))

{
    "predictedClassNumbers": [
        0,
        56,
        77,
        57,
        16,
        15,
        60,
        26,
        39,
        17
    ],
    "width": 290.0,
    "centerX": 135.0,
    "height": 476.0,
    "centerY": 373.0,
    "confidences": [
        0.9997019,
        9.041101e-05,
        4.9534065e-05,
        2.8544315e-05,
        1.7648663e-05,
        1.4100428e-05,
        8.065511e-06,
        7.890028e-06,
        7.1107734e-06,
        4.345046e-06
    ],
    "predictedClasses": [
        "person",
        "chair",
        "teddy bear",
        "sofa",
        "dog",
        "cat",
        "diningtable",
        "handbag",
        "bottle",
        "horse"
    ]
}


### Save Detection

In [15]:
with open('detections/img-4.json', 'w') as outfile:  
    json.dump(detections, outfile, indent=4, sort_keys=True)

## Visualize detections

1: [person, 0.9997]
2: [chair, 0.9906]
3: [person, 0.99942]
4: [person, 0.99956]
5: [diningtable, 0.99665]
6: [person, 0.99606]
7: [person, 0.9842]
8: [chair, 0.99833]


In [12]:
image = skil.utils.yolo.annotate_image(cv2.imread("images/img-4.jpg"), detections)

cv2.imwrite('images/annotated.jpg', image)

plt.figure(figsize=(9,9))
plt.imshow(cv2.cvtColor(image, cv2.COLOR_BGR2RGB))
plt.show()
image.shape

for k, detection in enumerate(detections['objects']):
    predicted = detection['predictedClasses'][0]
    confidence = detection['confidences'][0]
    
    print('{}: [{}, {:.5}]'.format(k+1, predicted, confidence))